In [4]:
import pandas as pd
file_path = 'ratings_Beauty.csv'  
df = pd.read_csv(file_path, nrows=10)
df.head()


,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split as surprise_train_test_split
from surprise.accuracy import rmse
from collections import defaultdict


data_path = "ratings_Beauty.csv"
columns = ["UserId", "ProductId", "Rating", "Timestamp"]
df = pd.read_csv(data_path, names=columns, header=0)

df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

# Ensure ratings are within valid range (1-5)
df = df[(df['Rating'] >= 1) & (df['Rating'] <= 5)]


reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['UserId', 'ProductId', 'Rating']], reader)
trainset, testset = surprise_train_test_split(data, test_size=0.2, random_state=42)


model = SVD()
model.fit(trainset)

predictions = model.test(testset)
evaluation_result = rmse(predictions)


def get_top_n(predictions, n=10):
    """Return the top-N recommendations for each user."""
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# Generate top-10 recommendations for each user
top_n_recommendations = get_top_n(predictions, n=10)

sample_user_id = list(top_n_recommendations.keys())[:5]
for user_id in sample_user_id:
    print(f"Recommendations for User {user_id}:")
    for product_id, rating in top_n_recommendations[user_id]:
        print(f"  Product: {product_id}, Estimated Rating: {rating:.2f}")
    print()


RMSE: 1.2476
Recommendations for User A171X776AY64AA:
  Product: B006UJ255I, Estimated Rating: 4.28

Recommendations for User A1T43BYE3IYTW2:
  Product: B00A5XWKXM, Estimated Rating: 4.49
  Product: B00267AETM, Estimated Rating: 4.40

Recommendations for User A8RUPITWLM2A5:
  Product: B000V3URNQ, Estimated Rating: 3.99

Recommendations for User A2BPB05QNCE00B:
  Product: B006L6A06Q, Estimated Rating: 3.75
  Product: B002MO3CG6, Estimated Rating: 3.74
  Product: B00106AC06, Estimated Rating: 3.33

Recommendations for User ASLM3QEVFGKPM:
  Product: B007H4HP7S, Estimated Rating: 4.41

